In [277]:
from math import radians, sqrt, sin, cos
import random
import torch as trch
import numpy as np

x_goal = 1.2
y_goal = 0.8
z_goal = 0.5

t_1_int = t_1 = radians(-90)
d_2_int = d_2 = 0.5
d_3_int = d_3 = 1
t_4_int = t_4 = radians(-90)
t_5_int = t_5 = radians(90)

# Constant
t_6 = radians(40)
d_6 = .2

#Arbitrary
d_1 = 3
threshold = 0.001


Part a

In [278]:
# Changing things
x = (cos(t_1) * cos(t_4) * sin(t_5) * d_6) - (sin(t_1) * cos(t_5) * d_6) - (sin(t_1) * d_3)
y = (sin(t_1) * cos(t_4) * sin(t_5) * d_6) + (cos(t_1) * cos(t_5) * d_6) + (cos(t_1) * d_3)
z = (sin(t_4) * sin(t_5) * d_6) + d_1 + d_2

def dist(x1, x2, y1, y2, z1, z2):
    return sqrt((x2-x1)**2 + (y2-y1)**2 + (z2-z1)**2)

n = 1
while(n > 0):
    
    change = 0
    
    factor = dist(x, x_goal, y, y_goal, z, z_goal) * 2
    t_1_diff = (random.random()-0.5) * factor
    t_4_diff = (random.random()-0.5) * factor
    t_5_diff = (random.random()-0.5) * factor
    d_2_diff = (random.random()-0.5) * factor
    d_3_diff = (random.random()-0.5) * factor

    t_1_temp = t_1 - t_1_diff
    t_4_temp = t_4 - t_4_diff
    t_5_temp = t_5 - t_5_diff
    d_2_temp = d_2 - d_2_diff
    d_3_temp = d_3 - d_3_diff

    x_tmp = (cos(t_1_temp) * cos(t_4_temp) * sin(t_5_temp) * d_6) - (sin(t_1_temp) * cos(t_5_temp) * d_6) - (sin(t_1_temp) * d_3_temp)
    y_tmp = (sin(t_1_temp) * cos(t_4_temp) * sin(t_5_temp) * d_6) + (cos(t_1_temp) * cos(t_5_temp) * d_6) + (cos(t_1_temp) * d_3_temp)
    z_tmp = (sin(t_4_temp) * sin(t_5_temp) * d_6) + d_1 + d_2_temp

    if abs(x_tmp - x_goal) < threshold and abs(y_tmp - y_goal) < threshold and abs(z_tmp - z_goal) < threshold:
        change = sum([abs(initial - temporary) for initial, temporary in zip((t_1_int, t_4_int, t_5_int, d_2_int, d_3_int), (t_1_temp, t_4_temp, t_5_temp, d_2_temp, d_3_temp))])
        print("success")
        print(f"Final XYZ {x_tmp:.4}, {y_tmp:.4}, {z_tmp:.4},")
        print(f"Final thetas {t_1_temp=:.4} {t_4_temp=:.4} {t_5_temp=:.4} {d_2_temp=} {d_3_temp=}")
        print("num iter {}".format(n))
        print('Change: ', change)
        break
    
    if dist(x_goal, x, y_goal, y, z_goal, z) < dist(x_goal, x_tmp, y_goal, y_tmp, z_goal, z_tmp):
        pass
    else:
        t_1 = t_1_temp 
        t_4 = t_4_temp
        t_5 = t_5_temp
        d_2 = d_2_temp
        d_3 = d_3_temp

        x = x_tmp
        y = y_tmp
        z = z_tmp

    n += 1

success
Final XYZ 1.2, 0.7999, 0.4995,
Final thetas t_1_temp=2.267 t_4_temp=2.462 t_5_temp=1.542 d_2_temp=-2.626208724917287 d_3_temp=-1.4390893231625133
num iter 104
Change:  13.464280212649161


Part b

In [279]:
x_goal = 1.2
y_goal = 0.8
z_goal = 0.5

t_1_int = t_1 = radians(0)
d_2_int = d_2 = 0.2
d_3_int = d_3 = .3
t_4_int = t_4 = radians(-90)
t_5_int = t_5 = radians(90)

# Constant
t_6 = radians(40)
d_6 = .2

#Arbitrary
d_1 = 3
threshold = 0.0001

In [280]:
class NN(trch.nn.Module):
    def __init__(self):
        super(NN, self).__init__()
        self.layer1 = trch.nn.Linear(8, 8)
        self.layer2 = trch.nn.Linear(8, 5)
    
    def forward(self, input):
        layer1_output = self.layer1(input)

        layer1_relu = trch.nn.functional.relu(layer1_output)

        output = self.layer2(layer1_relu)

        return output

In [281]:
class Agent(object):
    def __init__(self):
        self.device = trch.device("cuda:0" if trch.cuda.is_available() else "cpu")
        self.model = NN().to(self.device)
        self.optimizer = trch.optim.Adam(self.model.parameters(), lr=3e-3)
        self.loss_function = trch.nn.MSELoss()

    def act(self, state):
        
        state = np.array(state, dtype=np.float64)
        state = trch.from_numpy(state).unsqueeze(0).to(self.device)
        movement: trch.Tensor = self.model(state.float()).cpu().tolist()
        return movement


In [282]:
def dist(x1, x2, y1, y2, z1, z2):
    return sqrt((x2-x1)**2 + (y2-y1)**2 + (z2-z1)**2)

In [283]:

def getPosition(positional_info):
    t_1, d_2, d_3, t_4, t_5 = positional_info
    
    x = (cos(t_1) * cos(t_4) * sin(t_5) * d_6) - (sin(t_1) * cos(t_5) * d_6) - (sin(t_1) * d_3)
    y = (sin(t_1) * cos(t_4) * sin(t_5) * d_6) + (cos(t_1) * cos(t_5) * d_6) + (cos(t_1) * d_3)
    z = (sin(t_4) * sin(t_5) * d_6) + d_1 + d_2

    return (x, y, z)

start_positional_info = t_1, d_2, d_3, t_4, t_5

start_x, start_y, start_z = getPosition(start_positional_info)

def getReward(positional_info, steps):
    
    try:
        x, y, z = getPosition(positional_info)
    except ValueError:
        return -1000, True

    distance_to_start = dist(x, start_x, y, start_y, z, start_z)
    distance_to_end = dist(x, x_goal, y, y_goal, z, z_goal)
    reward = (1/distance_to_start) + (10/distance_to_end)
    return (reward, True if distance_to_end < .01 else False)

In [284]:
agent = Agent()

for i in range(1, 1001):
    done = False
    total_reward = 0
    steps = 0

    while not done:
        steps += 1

        current_positional_info = t_1, d_2, d_3, t_4, t_5
        x_current, y_current, z_current = getPosition(current_positional_info)
        state = np.array((t_1, d_2, d_3, t_4, t_5, x_current, y_current, z_current)) 
        t_1_change, d_2_change, d_3_change, t_4_change, t_5_change = agent.act(state)[0]

        t_1 += t_1_change
        d_2 += d_2_change
        d_3 += d_3_change
        t_4 += t_4_change
        t_5 += t_5_change

        current_positional_info = t_1, d_2, d_3, t_4, t_5

        reward, done = getReward(current_positional_info, steps)
        if (steps > 1000):
            total_reward = -1000
            break

        total_reward += reward

    total_reward /= steps

    loss = agent.loss_function()

    agent.optimizer.zero_grad()
    loss.backward()
    agent.optimizer.step()

    if (i % 50 == 0):
        print(f"{i=}: {total_reward=:<5.3}, {steps=:<5}")








i=50: total_reward=-0.999, steps=1001 
i=100: total_reward=-0.999, steps=1001 
i=150: total_reward=-0.999, steps=1001 
i=200: total_reward=-0.999, steps=1001 
i=250: total_reward=-0.999, steps=1001 
i=300: total_reward=-0.999, steps=1001 
i=350: total_reward=-0.999, steps=1001 
i=400: total_reward=-0.999, steps=1001 
i=450: total_reward=-0.999, steps=1001 
i=500: total_reward=-0.999, steps=1001 
i=550: total_reward=-0.999, steps=1001 
i=600: total_reward=-0.999, steps=1001 
i=650: total_reward=-0.999, steps=1001 
i=700: total_reward=-0.999, steps=1001 
i=750: total_reward=-0.999, steps=1001 
i=800: total_reward=-0.999, steps=1001 
i=850: total_reward=-0.999, steps=1001 
i=900: total_reward=-0.999, steps=1001 
i=950: total_reward=-0.999, steps=1001 
i=1000: total_reward=-0.999, steps=1001 
